In [1]:
import numpy as np

Define the end goal, that we want to achive in tensorflow:

In [2]:
np_pred = np.random.randn(4, 128)
np_true = np.array([[1],[1],[2],[3]])
print(np_pred.shape, np_true.shape)

(4, 128) (4, 1)


The goal is to comute the L2 distance between all permutations of the embeddings (rows of np_pred):

In [3]:
dist_pos = []
dist_neg = []
for i in range(np_pred.shape[0]-1):
    for j in range(i+1, np_pred.shape[0]):
        if np_true[i] == np_true[j]:
            dist_pos.append( np.linalg.norm(np_pred[i]-np_pred[j]) )
            print(f"Pos: {np.linalg.norm(np_pred[i]-np_pred[j])}")
        else:
            dist_neg.append( np.linalg.norm(np_pred[i]-np_pred[j]) )
            print(f"Neg: {np.linalg.norm(np_pred[i]-np_pred[j])}")

print(np.mean(dist_pos), np.mean(dist_neg))

Pos: 17.225099945474028
Neg: 18.015576579456305
Neg: 15.475763090014768
Neg: 17.257324233392087
Neg: 14.489370891432992
Neg: 15.764187603936438
17.225099945474028 16.200444479646517


Now lets go for a tensorflow implementation which will require posing this computation into a tensor opperation:

In [4]:
import tensorflow as tf

In [5]:
tf_pred = tf.constant(np_pred)
tf_true = tf.constant(np_true)
print(tf_pred.shape, tf_true.shape)

(4, 128) (4, 1)


In [6]:
tmp = tf.broadcast_to(tf_pred, (tf_pred.shape[0], tf_pred.shape[0], tf_pred.shape[1]))
tmp.shape

TensorShape([4, 4, 128])

In [7]:
difference_tensor = tmp - tf.transpose( tmp, [1,0,2] )
difference_tensor.shape

TensorShape([4, 4, 128])

In [8]:
pred_norm = tf.norm(difference_tensor, axis=2)

Nice! Now we need to get our maps for positive and negative examples:

In [76]:
tmp_true = tf.broadcast_to(tf_true, (tf_true.shape[0], tf_true.shape[0]))

true_diff =tf.cast( tmp_true == tf.transpose(tmp_true, [1,0]), tf.float64)
#Remove the diagonal entries:
true_diff = tf.linalg.set_diag(true_diff, tf.cast(tf.broadcast_to([0], [true_diff.shape[0]]), tf.double) )
true_diff = tf.linalg.band_part(true_diff, 0, -1) #Remove the symmetric part below the diagonal

#Do the same for the negative examples:
neg_diff = tf.cast( tmp_true != tf.transpose(tmp_true, [1,0]), tf.float64)
neg_diff = tf.linalg.band_part(neg_diff, 0, -1)

In [77]:
pos_distance = tf.boolean_mask(pred_norm, tf.cast(true_diff, tf.bool))
neg_distance = tf.boolean_mask(pred_norm, tf.cast(neg_diff, tf.bool))
pos_distance, neg_distance

(<tf.Tensor: shape=(1,), dtype=float64, numpy=array([17.22509995])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=array([18.01557658, 15.47576309, 17.25732423, 14.48937089, 15.7641876 ])>)

In [78]:
tf.reduce_sum(pos_distance)/pos_distance.shape[0], tf.reduce_sum(neg_distance)/neg_distance.shape[0]

(<tf.Tensor: shape=(), dtype=float64, numpy=17.225099945474028>,
 <tf.Tensor: shape=(), dtype=float64, numpy=16.200444479646514>)

Nice ... slightly complicated, but thats tensorflow...

# Pessimistic Metric

Averageing the distance of positive and negative examples seems to be wway too optimistic. In fact, the loss function does try to find the hardest negative examples (short distance) and the worst positive example (largest distance). Lets try to find them with tensorflow:

In [79]:
max_pos_dist = tf.reduce_max(pos_distance)
max_pos_dist

<tf.Tensor: shape=(), dtype=float64, numpy=17.225099945474028>

In [80]:
min_neg_dist = tf.reduce_min( neg_distance )
min_neg_dist

<tf.Tensor: shape=(), dtype=float64, numpy=14.489370891432992>

# All to All - Metric

One other metric might be to compare the distances of all positive and all negative examples (pairs) and return the mean of the combined scores (each being 1 if the positive distance was smaller then the negative distance for each example):

In [82]:
tf.broadcast_to(pos_distance, (neg_distance.shape[0], pos_distance.shape[0] ))

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[17.22509995],
       [17.22509995],
       [17.22509995],
       [17.22509995],
       [17.22509995]])>

In [85]:
tf.broadcast_to( neg_distance, (pos_distance.shape[0], neg_distance.shape[0]) )

<tf.Tensor: shape=(1, 5), dtype=float64, numpy=array([[18.01557658, 15.47576309, 17.25732423, 14.48937089, 15.7641876 ]])>

Unfortunately we have only one positive example. So lets make up a second one, to check if it works correctely:

In [93]:
pos = tf.constant([17.22, 15.24], dtype=tf.float64)
neg = neg_distance

In [98]:
t_pos = tf.broadcast_to(pos, (neg_distance.shape[0], pos.shape[0]))

In [99]:
t_neg = tf.transpose(tf.broadcast_to( neg_distance, (pos.shape[0], neg_distance.shape[0]) ))

In [100]:
t_pos < t_neg

<tf.Tensor: shape=(5, 2), dtype=bool, numpy=
array([[ True,  True],
       [False,  True],
       [ True,  True],
       [False, False],
       [False,  True]])>

In [101]:
t_pos, t_neg

(<tf.Tensor: shape=(5, 2), dtype=float64, numpy=
 array([[17.22, 15.24],
        [17.22, 15.24],
        [17.22, 15.24],
        [17.22, 15.24],
        [17.22, 15.24]])>,
 <tf.Tensor: shape=(5, 2), dtype=float64, numpy=
 array([[18.01557658, 18.01557658],
        [15.47576309, 15.47576309],
        [17.25732423, 17.25732423],
        [14.48937089, 14.48937089],
        [15.7641876 , 15.7641876 ]])>)

In [103]:
t_comp = t_pos < t_neg

In [109]:
tf.reduce_sum(tf.cast(t_comp, tf.int64))/(t_comp.shape[0]*t_comp.shape[1])

<tf.Tensor: shape=(), dtype=float64, numpy=0.6>